In [1]:
import warnings
warnings.filterwarnings('ignore')
from glob import glob 
import os 
import random 

import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from PIL import Image 
import cv2


import torch 
import torch.nn as nn 
import torch.nn.functional as F 
from torch.utils.data import Dataset,DataLoader
import torchvision.transforms as transforms 
import torchvision
import timm 
import wandb

from src.data.augmentation import *
from src.data.factory import create_dataset,create_dataloader
from src.options import Options
from src.models import Model 
from src.loss_function import STPMLoss
from src.train import AverageMeter,torch_seed,train_epoch,valid_epoch
from src.callback import Callbacks

In [2]:
def run(cfg):
    torch_seed(cfg['seed'])
    
    # build train,test loader 
    trainset,testset = create_dataset(cfg)
    train_loader = create_dataloader(trainset,
                                    cfg['Batchsize'],
                                    shuffle=True)
    test_loader = create_dataloader(testset,
                                    cfg['Batchsize'],
                                    shuffle=False)

    # build a model, criterion and optimizer 
    model = Model(cfg['modeltype']).to(cfg['device'])
    criterion = STPMLoss()
    optimizer = __import__('torch.optim', fromlist='optim').__dict__['Adam'](model.parameters(), lr=cfg['lr'],betas=(cfg['beta1'],0.999))
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,T_max=cfg['nepochs'])
    
    print('All loaded, Training start')
    fit(model,train_loader,test_loader,criterion,optimizer,scheduler,cfg)
    
def fit(model,train_loader,test_loader,criterion,optimizer,scheduler,cfg):
    
    callbacks = Callbacks(cfg)
    
    total_loss = {} 
    total_loss['train'] = [] 
    total_loss['valid'] = [] 
    
    for epoch in range(cfg['nepochs']):
        train_loss = train_epoch(model,train_loader,criterion,optimizer,cfg)
        valid_loss = valid_epoch(model,test_loader,criterion,cfg)

        
        total_loss['train'].append(train_loss)
        total_loss['valid'].append(valid_loss)
        scheduler.step()
        
        log = {'Epoch' : epoch,
               'train_loss' : train_loss,
               'valid_loss' : valid_loss,
               'learing_rate' : optimizer.param_groups[0]['lr']}
        
        #check point 
        callbacks.epoch(model,log)
    callbacks.epoch(model,log,'last')

In [88]:
cfg = Options().to_dict()            
run(cfg)

All loaded, Training start
 
Epoch : 0
 Train loss : 3.963 | Valid loss : 4.562
Best model saved at 0
 
Epoch : 1
 Train loss : 2.398 | Valid loss : 4.314
Best model saved at 1
 
Epoch : 2
 Train loss : 1.584 | Valid loss : 4.110
Best model saved at 2
 
Epoch : 3
 Train loss : 1.158 | Valid loss : 3.853
Best model saved at 3
 
Epoch : 4
 Train loss : 0.913 | Valid loss : 3.652
Best model saved at 4
 
Epoch : 5
 Train loss : 0.774 | Valid loss : 3.568
Best model saved at 5
 
Epoch : 6
 Train loss : 0.693 | Valid loss : 3.453
Best model saved at 6
 
Epoch : 7
 Train loss : 0.614 | Valid loss : 3.337
Best model saved at 7
 
Epoch : 8
 Train loss : 0.553 | Valid loss : 3.274
Best model saved at 8
 
Epoch : 9
 Train loss : 0.526 | Valid loss : 3.205
Best model saved at 9
 
Epoch : 10
 Train loss : 0.479 | Valid loss : 3.173
Best model saved at 10
 
Epoch : 11
 Train loss : 0.451 | Valid loss : 3.126
Best model saved at 11
 
Epoch : 12
 Train loss : 0.416 | Valid loss : 3.059
Best model save

KeyboardInterrupt: 